In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 4.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.9 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers import AutoTokenizer


In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
815482 * 0.

733933.8

In [ ]:
from datasets import load_dataset
from datasets import get_dataset_split_names
dataset = load_dataset("LucasThil/miniwobplusplus_ready", split='train[:733933]')
#dataset = dataset['train'][100]
get_dataset_split_names("LucasThil/miniwobplusplus_ready")

['train']

In [ ]:
dataset_eval = load_dataset("LucasThil/miniwobplusplus_ready", split='train[733933:]')


In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["episodes"], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/734 [00:00<?, ?ba/s]

In [ ]:
eval_dataset = dataset_eval.map(tokenize_function, batched=True)

  0%|          | 0/82 [00:00<?, ?ba/s]

In [ ]:
#len(tokenized_datasets['train'])
#import torch

# Let's now split up the data into train and validation sets
#n = int(0.9*len(tokenized_datasets['train']))
#train_data = tokenized_datasets['train'][:n]
#val_data = tokenized_datasets['train'][n:]

In [ ]:
tokenized_datasets = tokenized_datasets.shuffle(seed=42)
eval_dataset = eval_dataset.shuffle(seed=42)

In [ ]:
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(100))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(100))

In [ ]:
from transformers import AutoModelForSequenceClassification
# We found 2947 unique labels to predict which are a combination between actions and ref of element
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2947)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

In [ ]:
metric = evaluate.load('accuracy')

In [ ]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/test_save_path",
    evaluation_strategy="epoch",
    overwrite_output_dir=True,
    num_train_epochs=1,
    logging_steps=1000,
    save_steps=1000,
    resume_from_checkpoint=True,
    push_to_hub=True,
    )

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/LucasThil/test_save_path into local empty directory.


Download file pytorch_model.bin:   0%|          | 23.0k/413M [00:00<?, ?B/s]

Download file runs/Feb13_17-58-42_e23081260953/events.out.tfevents.1676311152.e23081260953.105.0: 100%|#######…

Download file runs/Feb13_17-58-42_e23081260953/1676311152.9009502/events.out.tfevents.1676311152.e23081260953.…

Clean file runs/Feb13_17-58-42_e23081260953/events.out.tfevents.1676311152.e23081260953.105.0:  22%|##1       …

Download file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

Clean file runs/Feb13_17-58-42_e23081260953/1676311152.9009502/events.out.tfevents.1676311152.e23081260953.105…

Clean file training_args.bin:  30%|##9       | 1.00k/3.37k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/413M [00:00<?, ?B/s]

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: episodes, actions. If episodes, actions are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 13
  Number of trainable parameters = 110576515


ValueError: ignored

In [ ]:
print(small_train_dataset[:100])

{'episodes': ['\'{action: keypress, ref: 21}{action: keypress, ref: 21}{action: keydown, ref: 21}{action: keydown, ref: 21}{action: keyup, ref: 21}{action: keyup, ref: 21}{action: keypress, ref: 21}{action: keypress, ref: 21}{action: keydown, ref: 21}{action: keydown, ref: 21}{action: click, ref: 21}{action: mouseup, ref: 21}{action: mouseup, ref: 21}{action: mousedown, ref: 1}{action: mousedown, ref: 1}{action: click, ref: 1}{action: mouseup, ref: 1}{action: mouseup, ref: 1}{action: mousedown, ref: 15}{action: mousedown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{action: keyup, ref: 15}{action: keyup, ref: 15}{action: keypress, ref: 15}{action: keypress, ref: 15}{action: keydown, ref: 15}{action: keydown, ref: 15}{a